In [1]:
import pandas as pd
import itertools
from gurobipy import *

In [2]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
production = pd.read_excel("base_case_data.xlsx", header = 0, index_col=0, sheet_name = "production")

setup = pd.read_excel("base_case_data.xlsx", header = 0, index_col=0, sheet_name = "setup")

demand = pd.read_excel("base_case_data.xlsx", header = 0, index_col=0, sheet_name = "demand")

In [3]:
# set up decision variables
# reference: https://stackoverflow.com/questions/1953194/permutations-of-two-lists-in-python
# part1, part2, part3...
part_names = list(setup.columns)

# machine1, machine2, machine3...
machine_names = list(setup.index)

# Week1, Week2, Week3...
week_list = []
for i in range(12):
    week_list.append("Week" + str(i+1))

In [4]:
# variable list
var_list_per_week = []
for i in range(len(demand)):
    for r in itertools.product(machine_names, part_names):
        if setup.loc[r[0], r[1]] != 0:
            var_list_per_week.append("Week" + str(i+1) + "_" + r[0]+ "_" +r[1]) 
# var_list_per_week
inventory_variable_list = []
for w in week_list:
    for p in part_names:
        name = w+"_"+p
        inventory_variable_list.append(name)
# inventory_variable_list

In [5]:
# coefficient
setup_co = {}
for v in var_list_per_week:
    machine = v.split("_")[1]
    part = v.split("_")[2]
    setup_co[v] = setup.loc[machine, part]
# setup_co

In [6]:
# right constraints
cumulative_right_eq = {}
for i in demand.index[0:12]:
    for name in part_names:
        if name in cumulative_right_eq.keys():
            cumulative_right_eq[name] += demand.loc[i, name]
        else:
            cumulative_right_eq[name] = demand.loc[i, name]
# cumulative_right_eq

# demand for each week 
week_right_geq = {}
for i in range(12):
    week_demand = {}
    for name in part_names:
        week_demand[name] = demand.loc[i+1, name]
    week_right_geq["Week"+str(i+1)] = week_demand
# week_right_geq

# maximum regular working hours per week
regular_hour_right_leq = {}
for name in machine_names:
    regular_hour_right_leq[name] = 120
# constraint_right_eq

# maximum overtime hours per week
overtime_hour_right_leq = {}
for name in machine_names:
    overtime_hour_right_leq[name] = 48
    
# inventory constraint right hand side
inventory_right = {}
for w in week_list:
    part_dic = {}
    for p in part_names:
        part_dic[p] = 0
    inventory_right[w] = part_dic
# inventory_right

In [7]:
# Demand of each part: Done
cumulative_left_eq = {}
for p in part_names:
    production_co = {}
    for v in var_list_per_week:
        machine = v.split("_")[1]
        part = v.split("_")[2]
        if part == p:
            production_co[v] = production.loc[machine, part]
        else:
            production_co[v] = 0
    cumulative_left_eq[p] = production_co
# cumulative_left_geq

week_left_geq = {}
for w in week_list:
    week_dic = {}
    for p in part_names:
        part_dic = {}
        for v in var_list_per_week:
            week = v.split("_")[0]
            machine = v.split("_")[1]
            part = v.split("_")[2]
            if week == w and part == p:
                part_dic[v] = production.loc[machine, part]
            else:
                part_dic[v] = 0
        week_dic[p] = part_dic
    week_left_geq[w] = week_dic


# setup time for each machine per week
time_left_s_leq = {}
for w in week_list:
    week_dic = {}
    for m in machine_names:
        setup_dic = {}
        for v in var_list_per_week:
            week = v.split("_")[0]
            machine = v.split("_")[1]
            part = v.split("_")[2]
            if machine == m and week == w:
                setup_dic[v] = setup.loc[machine, part]
            else:
                setup_dic[v] = 0
        week_dic[m] = setup_dic
    time_left_s_leq[w] = week_dic

# working hours for each machine
working_left_leq = {}
for w in week_list:
    week_dic = {}
    for m in machine_names:
        production_dic = {}
        for v in var_list_per_week:
            week = v.split("_")[0]
            machine = v.split("_")[1]
            part = v.split("_")[2]
            if  machine == m and week == w:
                production_dic[v] = 1
            else:
                production_dic[v] = 0
        week_dic[m] = production_dic
    working_left_leq[w] = week_dic
# working_right_leq
# week_left_geq
demand_left = {}
for i in range(12):
    week_demand = {}
    for name in part_names:
        week_demand[name] = demand.loc[i+1, name]
    demand_left["Week"+str(i+1)] = week_demand
# demand_left

# inventory left side
inventory_left = {}
for w in week_list:
    dic = {}
    for p in part_names:
        part_dic = {}
        for v in inventory_variable_list:
            week = v.split("_")[0]
            part = v.split("_")[1]
            if w == "Week1":
                if week == "Week1" and part == p:
                    part_dic[v] = 1
                else:
                    part_dic[v] = 0
            elif w == "Week12" and part == p:
                if week == "Week11":
                    part_dic[v] = -1
                else:
                    part_dic[v] = 0
            else:
                number = int(w[4:])
                if week == "Week"+str(number-1) and part == p:
                    part_dic[v] = -1 
                elif week == w and part == p:
                    part_dic[v] = 1
                else:
                    part_dic[v] = 0
        dic[p] = part_dic
    inventory_left[w] = dic
# inventory_left

In [8]:
m=Model()
# set up variables
regular_production_levels = m.addVars(var_list_per_week, name="regular")
setup_levels = m.addVars(var_list_per_week, name="setup", vtype = GRB.BINARY)
inventory_levels = m.addVars(inventory_variable_list, name="inventory") 

# constraint
# cumulative production: DONE
eq_constraints_set = m.addConstrs((quicksum(setup_levels[v]*cumulative_left_eq[p][v]*regular_production_levels[v] for v in var_list_per_week)
                                   == cumulative_right_eq[p] for p in part_names ), name="eq_constraints");

# regular working hours per week: DONE
regular_leq = m.addConstrs((quicksum(working_left_leq[w][m][vp]*regular_production_levels[vp] for vp in var_list_per_week) + quicksum(time_left_s_leq[w][m][vs]*setup_levels[vs] for vs in var_list_per_week) 
                            <= regular_hour_right_leq[m] for m in machine_names for w in week_list), name="regular_leq");


# inventory constraint per week: TODO
production_per_week = {}
for w in week_list:
    week_dic = {}
    for p in part_names:
        week_dic[p] = quicksum(setup_levels[v]*week_left_geq[w][p][v]*regular_production_levels[v] for v in var_list_per_week)
    production_per_week[w] = week_dic

# week1 constraint    
week1_constr = m.addConstrs((demand_left["Week1"][p] + quicksum(inventory_left["Week1"][p][i]*inventory_levels[i] for i in inventory_variable_list) - production_per_week["Week1"][p]
                             == 0 for p in part_names), name="week1_inventory");

Academic license - for non-commercial use only


In [9]:
# week2 - week 12 constraint
other_week_constr = m.addConstrs((demand_left[week_list[i+1]][p] + quicksum(inventory_left[week_list[i+1]][p][j]*inventory_levels[j] for j in inventory_variable_list) - production_per_week[week_list[i+1]][p]
                             == 0 for p in part_names for i in range(len(week_list)-1)), name="other_week_inventory");

# inventory >= 0
inventory_constr = m.addConstrs((inventory_levels[i]>= 0 for i in inventory_variable_list), name="positive_inventory"); 

In [10]:
# objective function: minimize total cost = regular labour + setup + inventory
obj = quicksum(100*regular_production_levels[vp] for vp in var_list_per_week) + quicksum(100*setup_levels[vs]*setup_co[vs] for vs in var_list_per_week) + quicksum(10*inventory_levels[i] for i in inventory_variable_list)
m.update()
# minimize objective
m.setObjective(obj, GRB.MINIMIZE)
m.update()
# optimize model
m.optimize()
overtime_hours =0
#reference URL:www.gurobi.com/documentation/8.1/quickstart_mac/py_reporting_results_attri.html
for v in m.getVars():
    if v.x != 0:
        name = v.varName
        n1 = name.split("[")[0]
        n2 = name.split("[")[-1][:-1]
        if n1 == "setup":
            value = setup_co[n2]*v.x
        elif n1 == "overtime":
            value = v.x
            overtime_hours += v.x
        else:
            # value = v.x*production_co[n2]
            value = v.x
        print('%s %g' % (v.varName, value))
print('Obj: %g' % m.objVal)
print('total overtime hours: %g' % overtime_hours)

Optimize a model with 120 rows, 300 columns and 300 nonzeros
Model has 65 quadratic constraints
Variable types: 180 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
  QRHS range       [2e+03, 5e+04]
Presolve removed 60 rows and 5 columns
Presolve time: 0.00s
Presolved: 485 rows, 415 columns, 1430 nonzeros
Variable types: 295 continuous, 120 integer (120 binary)

Explored 1 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


AttributeError: b"Unable to retrieve attribute 'x'"

## Comment
Without overtime production, we got infeasible solution. So we still need overtime production to meet the demands for each week. 